In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd

sheet_url = "https://docs.google.com/spreadsheets/d/1pEJVFBYfT7gtRrvqLxbLIr8rursXXnUAk-42NBAhBEA/export?format=csv"
df = pd.read_csv(sheet_url)
df.shape

(29, 10)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   model_name    29 non-null     object
 1   model_sku     29 non-null     object
 2   price         29 non-null     object
 3   brand         29 non-null     object
 4   detail_url    29 non-null     object
 5   images        29 non-null     object
 6   specs         29 non-null     object
 7   category_url  29 non-null     object
 8   product_url   29 non-null     object
 9   image_url     29 non-null     object
dtypes: object(10)
memory usage: 2.4+ KB


In [3]:
filtered_df = df[['model_name', 'model_sku', 'product_url']]
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   model_name   29 non-null     object
 1   model_sku    29 non-null     object
 2   product_url  29 non-null     object
dtypes: object(3)
memory usage: 828.0+ bytes


In [ ]:
# 필수 패키지 설치
!pip install pandas requests beautifulsoup4 selenium webdriver-manager --upgrade
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently 

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
^C


In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. 크롤러(Driver) 관리 클래스
class WebDriverManager:
    def __init__(self):
        options = Options()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(options=options)

    def quit(self):
        self.driver.quit()

class ProductPDFPage:
    def __init__(self, driver, url):
        self.driver = driver
        self.url = url
        self.wait = WebDriverWait(self.driver, 20)

    def open(self):
        self.driver.get(self.url)
        time.sleep(1)

    def click_last_tab(self):
        try:
            menu_list = self.driver.find_element(By.CSS_SELECTOR, 'div.contents-menu-list')
            li_items = menu_list.find_elements(By.TAG_NAME, 'li')
            if not li_items:
                print("    ❌ li 항목 없음")
                return False

            last_tab = li_items[-1].find_element(By.TAG_NAME, 'a')
            self.driver.execute_script("arguments[0].scrollIntoView(true);", last_tab)
            time.sleep(1)

            # 🔥 JS 강제 클릭
            self.driver.execute_script("arguments[0].click();", last_tab)
            time.sleep(1)
            return True

        except Exception as e:
            print(f"    ❌ 다운로드 탭 클릭 실패: {e}")
            return False


    def get_pdf_links(self):
        pdf_links = []
        try:
            if self.click_last_tab():
                # 다운로드 섹션 등장 대기
                self.wait.until(EC.presence_of_element_located((By.ID, "download01_1")))
                ul = self.driver.find_element(By.CSS_SELECTOR, "#download01_1 ul.download-list")
                links = ul.find_elements(By.CSS_SELECTOR, 'a[data-type="PDF"]')
                for link in links:
                    href = link.get_attribute("href")
                    if href:
                        pdf_links.append(href)
        except Exception as e:
            print(f"    ❌ PDF 링크 수집 실패: {e}")
        return pdf_links


In [12]:
def collect_all_pdf_links(product_urls):
    manager = WebDriverManager()
    driver = manager.driver
    results = []

    for url in product_urls:
        print(f"🔍 Processing: {url}")
        try:
            page = ProductPDFPage(driver, url)
            page.open()
            links = page.get_pdf_links()
            print(f"✅ Found {len(links)} PDF links")
        except Exception as e:
            print(f"❌ Error: {e}")
            links = []
        results.append(links)

    manager.quit()
    return results


In [15]:
filtered_df['pdf_url'] = collect_all_pdf_links(filtered_df['product_url'])
print(filtered_df[['product_url', 'pdf_url']].head())

🔍 Processing: https://www.lge.co.kr/wash-tower/w2520wzm
✅ Found 1 PDF links
🔍 Processing: https://www.lge.co.kr/wash-tower/wa2525egzf
✅ Found 1 PDF links
🔍 Processing: https://www.lge.co.kr/wash-tower/wa2525iihf
✅ Found 1 PDF links
🔍 Processing: https://www.lge.co.kr/wash-tower/wa2525ggzf
✅ Found 1 PDF links
🔍 Processing: https://www.lge.co.kr/wash-tower/wa2525gmzf
    ❌ PDF 링크 수집 실패: Message: 
Stacktrace:
#0 0x5b29d07e123a <unknown>
#1 0x5b29d028bab0 <unknown>
#2 0x5b29d02dd6f0 <unknown>
#3 0x5b29d02dd8e1 <unknown>
#4 0x5b29d032bb94 <unknown>
#5 0x5b29d03031cd <unknown>
#6 0x5b29d0328fee <unknown>
#7 0x5b29d0302f73 <unknown>
#8 0x5b29d02cfaeb <unknown>
#9 0x5b29d02d0751 <unknown>
#10 0x5b29d07a5b4b <unknown>
#11 0x5b29d07a9929 <unknown>
#12 0x5b29d078c929 <unknown>
#13 0x5b29d07aa4e8 <unknown>
#14 0x5b29d07710df <unknown>
#15 0x5b29d07ce8e8 <unknown>
#16 0x5b29d07ceac6 <unknown>
#17 0x5b29d07e0556 <unknown>
#18 0x79ccd659aac3 <unknown>

✅ Found 0 PDF links
🔍 Processing: https://www.lg

/tmp/ipython-input-15-2799035311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['pdf_url'] = collect_all_pdf_links(filtered_df['product_url'])


In [16]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   model_name   29 non-null     object
 1   model_sku    29 non-null     object
 2   product_url  29 non-null     object
 3   pdf_url      29 non-null     object
dtypes: object(4)
memory usage: 1.0+ KB


In [29]:
filtered_df[filtered_df['pdf_url'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

,model_name,model_sku,product_url,pdf_url
4,LG 트롬 AI 오브제컬렉션 워시타워,wa2525gmzf,https://www.lge.co.kr/wash-tower/wa2525gmzf,[]


In [31]:
import os
import requests
import re
from tqdm import tqdm

# 저장 경로
save_dir = "/content/drive/Othercomputers/My Mac/Workspaces/SKN14-3rd-3Team/webscraping/YiseoY/lg_manuals"
os.makedirs(save_dir, exist_ok=True)

# 🔧 파일명 안전하게
def sanitize_filename(text):
    return re.sub(r'[\\/*?:"<>|]', '_', text)

# ✅ 전역 순번 초기화
pdf_counter = 1

# ✅ 수집 및 저장 루프
for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
    model_name = sanitize_filename(str(row['model_name']).strip().replace(" ", "_"))
    model_sku  = sanitize_filename(str(row['model_sku']).strip())
    pdf_list = row['pdf_url']

    if not isinstance(pdf_list, list) or len(pdf_list) == 0:
        continue

    for pdf_url in pdf_list:
        try:
            response = requests.get(pdf_url, timeout=10)
            if response.status_code == 200:
                file_name = f"{model_name}_{model_sku}_manual_{pdf_counter:04d}.pdf"
                file_path = os.path.join(save_dir, file_name)
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f"✅ 저장됨: {file_name}")
                pdf_counter += 1
            else:
                print(f"⚠️ 응답 오류 ({response.status_code}): {pdf_url}")
        except Exception as e:
            print(f"❌ 다운로드 실패: {pdf_url} | 오류: {e}")

  3%|▎         | 1/29 [00:00<00:17,  1.63it/s]

✅ 저장됨: LG_트롬_오브제컬렉션_워시타워_w2520wzm_manual_0001.pdf


 10%|█         | 3/29 [00:01<00:07,  3.47it/s]

✅ 저장됨: LG_트롬_AI_오브제컬렉션_워시타워_wa2525egzf_manual_0002.pdf
✅ 저장됨: LG_트롬_AI_오브제컬렉션_워시타워_wa2525iihf_manual_0003.pdf


 21%|██        | 6/29 [00:01<00:03,  6.05it/s]

...

✅ 저장됨: LG_트롬_오브제컬렉션_건조기_rg18wnst_manual_0025.pdf
✅ 저장됨: LG_트롬_오브제컬렉션_건조기_rg20en_manual_0026.pdf


 93%|█████████▎| 27/29 [00:05<00:00,  4.93it/s]

✅ 저장됨: LG_트롬_오브제컬렉션_건조기_rg20vn_manual_0027.pdf


100%|██████████| 29/29 [00:06<00:00,  4.52it/s]

✅ 저장됨: LG_트롬_오브제컬렉션_건조기_rg20kn_manual_0028.pdf
✅ 저장됨: LG_트롬_오브제컬렉션_건조기_rg18wnwt_manual_0029.pdf
